# 9. rabbit and dogs

In [ ]:
import copy

In [ ]:
def direction():
    p1 = []
    p2 = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    p3 = [(1, 1), (1,-1), (-1, 1), (-1,-1)] + p2
    m = [
        [p1, p3, p2, p3, p1],
        [p3, p2, p3, p2, p3],
        [p1, p3, p2, p3, p1]
    ]
    return m

In [ ]:
def get_moves(_pos):
    r = None
    m = direction()
    if (_pos["turn"] % 2) == 1: #dogs turn
        dogs_list = []
        for x, y in _pos["dogs"]:
            for ad_x, ad_y in m[y][x]:
                if ad_x < 0: #dogs can't move to back
                    continue
                (new_x, new_y) = (x + ad_x, y + ad_y)
                if new_x < 0 or len(m[0]) <= new_x or new_y < 0 or len(m) <= new_y:
                    continue
                v = m[new_y][new_x]
                if v == []:
                    continue
                if (new_x, new_y) in _pos["dogs"] + [_pos["rabbit"]]:
                    continue
                new_dogs = copy.deepcopy(_pos["dogs"])
                new_dogs.remove((x, y))
                new_dogs.append((new_x, new_y))
                dogs_list.append(new_dogs)
        r = dogs_list
    else: #rabbit turn
        rabbit_list = []
        x, y = _pos["rabbit"]
        for ad_x, ad_y in m[y][x]:
            (new_x, new_y) = (x + ad_x, y + ad_y)
            if new_x < 0 or len(m[0]) <= new_x or new_y < 0 or len(m) <= new_y:
                    continue
            v = m[new_y][new_x]
            if v == []:
                continue
            if (new_x, new_y) in _pos["dogs"] + [_pos["rabbit"]]:
                continue
            new_rabbit = (new_x, new_y)
            rabbit_list.append(new_rabbit)
        r = rabbit_list
    return r

In [ ]:
def next_turn(rabbit, dogs, turn):
    _pos = {
        "turn": turn + 1,
        "rabbit": copy.deepcopy(rabbit),
        "dogs": copy.deepcopy(dogs)
    }
    return _pos

In [ ]:
def to_tuple(_pos):
    return (_pos["turn"], _pos["rabbit"], tuple(sorted(_pos["dogs"])))

In [ ]:
def tree(root, _pos):
    if 30 < _pos["turn"]:
        root[to_tuple(_pos)] = { "gameset": 2, "next": [] }
        return 2 #rabbit win
    if _pos["rabbit"][0] <= min([ x for (x, y) in _pos["dogs"] ]):
        root[to_tuple(_pos)] = { "gameset": 2, "next": [] }
        return 2 #rabbit win
    is_dogs = True if (_pos["turn"] % 2) == 1 else False
    moves = get_moves(_pos)
    if is_dogs == False and moves == []:
        root[to_tuple(_pos)] = { "gameset": 1, "next": [] }
        return 1 #dogs win
    result = []
    for candi in moves:
        if is_dogs == True:
            p = next_turn(_pos["rabbit"], candi, _pos["turn"])
        else:
            p = next_turn(candi, _pos["dogs"], _pos["turn"])
        t = to_tuple(p)
        try:
            r = root[t]["gameset"]
            if r != -1:
                result.append((r, p))
                continue
        except:
            pass
        root.setdefault(t, { "gameset": -1, "next": [] })
        r = tree(root, p)
        result.append((r, p))
    if is_dogs == True:
        r = 1 if 0 < len([ w for w, p in result if w == 1 ]) else 2
    else:
        r = 2 if 0 < len([ w for w, p in result if w == 2 ]) else 1
    root[to_tuple(_pos)] = { "gameset": r, "next": copy.deepcopy(result) }
    return r

In [ ]:
def main():
    root = {}
    rabbits = [ (2, 0), (3, 0), (1, 1), (2, 1), (3, 1), (4, 1), (2, 2), (3, 2) ]
    for rabbit in rabbits:
        _pos = next_turn(rabbit, [ (1, 0), (0, 1), (1, 2) ], 0)
        tuple_pos = to_tuple(_pos)
        root.setdefault(tuple_pos, { "gameset": -1, "next": [] })
        r = tree(root, _pos)
        root[tuple_pos]["gameset"] = r
    return root

In [ ]:
table = main()

## Win/Lose rate

In [ ]:
counter = {}
for k in table.keys():
    name = "dogs" if 1 == table[k]["gameset"] else "rabbit"
    counter.setdefault(name, 0)
    counter[name] += 1
print(counter)

## Results from each initial position

In [ ]:
rabbits = [ (2, 0), (3, 0), (1, 1), (2, 1), (3, 1), (4, 1), (2, 2), (3, 2) ]
for rabbit in rabbits:
    _pos = next_turn(rabbit, [ (1, 0), (0, 1), (1, 2) ], 0)
    tuple_pos = to_tuple(_pos)
    name = "dogs" if 1 == table[tuple_pos]["gameset"] else "rabbit"
    print(str(rabbit) + ": " + name + " win")

## Print the best move

In [ ]:
def print_pos(_pos, gameset):
    _map = [
        [-1, 0, 0, 0, -1],
        [ 0, 0, 0, 0,  0],
        [-1, 0, 0, 0, -1]
    ]
    if gameset == 1:
        print("win: dogs " + "(turn." + str(_pos["turn"]) + ")")
    else:
        print("win: rabbit " + "(turn." + str(_pos["turn"]) + ")")
    rx, ry = _pos["rabbit"]
    _map[ry][rx] = 2 #rabbit
    for dx, dy in _pos["dogs"]:
        _map[dy][dx] = 1 #dogs
    for l in _map:
        line = ""
        for v in l:
            if v == -1:
                line += "-"
            if v == 0:
                line += "*"
            if v == 1:
                line += "D"
            if v == 2:
                line += "R"
        print(line)

In [ ]:
turn, rabbit, dogs = (5, (3, 1), [ (0, 1), (2, 1), (2, 2) ])
_pos = next_turn(rabbit, dogs, turn-1)
print_pos(_pos, table[to_tuple(_pos)]["gameset"])
print()
print("-------------------")
for result, _p in table[to_tuple(_pos)]["next"]:
    print()
    print_pos(_p, result)